In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

# 원본 데이터 로드
origin_df = pd.read_csv("./04_hashed.csv")

In [2]:
# IP 전처리
def transform_ip(ip): 
    groups = ip.split(".") 
    equalize_group_length = "".join( map( lambda group: group.zfill(3), groups )) 
    return equalize_group_length 

# Feature 전처리
def preprocess(df):
      
    # IP 전처리
    df['src_ip'] = df.src_ip.apply(lambda ip : transform_ip(ip))
    df['dst_ip'] = df.dst_ip.apply(lambda ip : transform_ip(ip))

    # country 전처리
    label_encoder = LabelEncoder()
    df['src_country'] = label_encoder.fit_transform(df['src_country'])
    df['dst_country'] = label_encoder.fit_transform(df['dst_country'])
    
    return df

In [3]:
# 모델 학습 및 추론
def model(input_df, model_name):
    # Rdate를 제외한 Feature 선택
    feature_preprocess_df = input_df[["src_ip", "dst_ip", "Proto", "src_port", "dst_port", "Action", "src_country", "dst_country"]]
    print('일하는중')
    # Isolation Forest 모델 선언 및 학습
    model = IsolationForest(random_state=0)
    model.fit(feature_preprocess_df)

    # 예측값 생성
    y_pred = model.predict(feature_preprocess_df)
    
    # 예측값 DataFrame화
    y_pred_df = pd.DataFrame(y_pred, columns=[f'{model_name}_y_pred'], index=feature_preprocess_df.index)
    # Feature 데이터와 예측값 concat
    result_df = pd.concat([feature_preprocess_df,y_pred_df],axis=1)
    return result_df, feature_preprocess_df, y_pred

In [4]:
# Rdate 종류
unique_time = origin_df['Rdate'].unique()

# Rdate를 제외한 나머지 Feature Scaling
preprocess_df = preprocess(origin_df.astype(str))
preprocess_df

,Rdate,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country
0,20210410000018.64,154058159102,103177012042,6,52897,445,2,121,224
1,20210410000018.64,154058159020,125066092196,6,60579,445,2,121,54
2,20210410000018.64,154058159164,117121178223,6,63831,445,2,121,224
3,20210410000018.64,154058159165,205034095097,6,55241,445,2,121,224
4,20210410000018.64,154058159102,093056164131,6,52898,445,2,121,224
...,...,...,...,...,...,...,...,...,...
5820305,20210411235934.81,012150252150,102116052100,6,64259,445,2,121,224
5820306,20210411235934.81,012150252150,167212157124,6,64261,445,2,121,46
5820307,20210411235934.81,145217002097,072192214054,6,25646,23,2,47,116
5820308,20210411235934.81,154058159164,185151199060,6,54633,445,2,121,159


In [ ]:
# window size 선언
window_size = 10

# 전체 데이터 탐지 결과
total_result_df = pd.DataFrame()

last_search_index = len(unique_time)-window_size
count = 0
model_name = 'isolation_forest'

for i in range(0, last_search_index, window_size):
    # window df 생성
    window_df = pd.DataFrame()
    print('test')
    # window size 만큼 Rdate 탐색
    for j in range(window_size):
        print(j)
        # 전체 데이터를 순서대로 확인하며, Rdate 종류에 해당하는 데이터를 추출해 window df에 추가
        select_time = preprocess_df[preprocess_df['Rdate']==unique_time[i+j]]
        window_df = pd.concat([window_df,select_time])
        
    # 모델 실행
    result_df, feature_df, y_pred = model(window_df, model_name)
    total_result_df = pd.concat([total_result_df, result_df])
    scatter_matrix(feature_df,
               c = y_pred,
               alpha=0.5,
               figsize=(20, 20),
               diagonal='kde')
    
    plt.show()
    
    count += window_size
    print(" @@@@@@@@@@@@@@@@@@@@ %d 개 Rdate 중, %d 개 확인 완료 @@@@@@@@@@@@@@@@@@@@ " % (len(unique_time), count))
    

test
0


In [ ]:
# 탐지 결과 파일 저장
origin_df[f'{model_name}_y_pred'] = total_result_df[f'{model_name}_y_pred']
origin_df.to_csv(f"{model_name}_result.csv", index=False)
origin_df